## Script pour enregistrer automatiquement des PDFs sur GALLICA à partir de titres de documents ##

* Mars 2021

**Ouverture du fichier texte composé de la liste de tous les idMoreau + tous les titres à chercher séparés par des tabs**

In [1]:
import os
import re
import urllib
import lxml.etree as etree

f = ""
f= open("lettres_titre_tronque.txt", "r", encoding='utf8')
lignes=f.readlines()
f.close()
f= open("sortie\liste_url_pdf_GALLICA_epistolaire.txt", "w", encoding='utf8') #Création du fichier texte de sortie
f.close()

In [2]:
lignes[0:5]

['192\tArrêt de la cour de parlement de Bordeaux, portant la justification de monsieur le prince sur le sujet des calomnies, inventées con\n',
 '200\t"Arrêt de la cour de parlement de Dijon, donné, les chambres assemblées, contre le cardinal Mazarin ; ensemble deux lettres dudit pa"\n',
 '272\tArrêts de la cour de parlement contre le libelle intitulé : Trés-humbles remontrances du parlement de Normandie , au semestre de se\n',
 '287\tArrêt dela cour de parlement, toutes les chambres assemblées, portant que le roi et la reine régente seront très-humblement supplié\n',
 '313\tArrêt de la cour de parlement et lettres patentes du roi pour la vente et distribution du pain à la livre dans Paris, à un prix rég\n']

**Nettoyage des titres pour les transformer en URL valides**

In [3]:
data = [re.sub("\(\w*\)", "", i) for i in lignes] #Suppression des articles entre parenthèses
data = [re.sub(',', '%20', i) for i in data]
data = [re.sub('\'', '%20', i) for i in data]
data = [re.sub(' ', '%20', i) for i in data]
data = [re.sub('é', '%C3%A9', i) for i in data]
data = [re.sub('à', '%C3%A0', i) for i in data]
data = [re.sub('è', '%C3%A8', i) for i in data]
data = [re.sub('ê', '%C3%AA', i) for i in data]
data = [re.sub('[^A-Za-z0-9%\t]+', '', i) for i in data]
data = [re.sub("\n", '', i) for i in data]

**Affichage du nombre de titres concernés et prévisualisation des titres nettoyés**

In [4]:
data[0:5]

['192\tArr%C3%AAt%20de%20la%20cour%20de%20parlement%20de%20Bordeaux%20%20portant%20la%20justification%20de%20monsieur%20le%20prince%20sur%20le%20sujet%20des%20calomnies%20%20invent%C3%A9es%20con',
 '200\tArr%C3%AAt%20de%20la%20cour%20de%20parlement%20de%20Dijon%20%20donn%C3%A9%20%20les%20chambres%20assembl%C3%A9es%20%20contre%20le%20cardinal%20Mazarin%20%20ensemble%20deux%20lettres%20dudit%20pa',
 '272\tArr%C3%AAts%20de%20la%20cour%20de%20parlement%20contre%20le%20libelle%20intitul%C3%A9%20%20Tr%C3%A9shumbles%20remontrances%20du%20parlement%20de%20Normandie%20%20%20au%20semestre%20de%20se',
 '287\tArr%C3%AAt%20dela%20cour%20de%20parlement%20%20toutes%20les%20chambres%20assembl%C3%A9es%20%20portant%20que%20le%20roi%20et%20la%20reine%20r%C3%A9gente%20seront%20tr%C3%A8shumblement%20suppli%C3%A9',
 '313\tArr%C3%AAt%20de%20la%20cour%20de%20parlement%20et%20lettres%20patentes%20du%20roi%20pour%20la%20vente%20et%20distribution%20du%20pain%20%C3%A0%20la%20livre%20dans%20Paris%20%20%C3%A0%20un%

**Affichage du nombre de titres concernés**

In [5]:
print("Nombre de titres à lancer dans l'API Gallica:")
print(len(data))

Nombre de titres à lancer dans l'API Gallica:
584


**Génération automatique des URLs pour l'API Gallica à partir des titres et enregistrement des 3 premiers résultats de la requête dans un fichier "idMoreau.xml"**

In [6]:
chaine1 = "https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22"
chaine2 = "%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5"
#Paramètres actuels de la requête API: titre de la mazarinade + publiée au XVIIe siècle + 3 premiers résultats
#Pour changer le nombre de résultats enregistrés, remplacer ""&maximumRecords=3" par le chiffre désiré


In [7]:
for d in data:
    idMoreau, titre=re.split("\t",d)
    url= chaine1 + titre + chaine2
    print(url)
    try:
        txt = urllib.request.urlretrieve(url, '%s.xml' %idMoreau)
    except:
        pass

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Arr%C3%AAt%20de%20la%20cour%20de%20parlement%20de%20Bordeaux%20%20portant%20la%20justification%20de%20monsieur%20le%20prince%20sur%20le%20sujet%20des%20calomnies%20%20invent%C3%A9es%20con%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Arr%C3%AAt%20de%20la%20cour%20de%20parlement%20de%20Dijon%20%20donn%C3%A9%20%20les%20chambres%20assembl%C3%A9es%20%20contre%20le%20cardinal%20Mazarin%20%20ensemble%20deux%20lettres%20dudit%20pa%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Arr%C3%AAts%20de%20la%20cour%20de%20parlement%20contre%20le%20libelle%20intitul%C3%A9%20%20Tr%C3%A9shumbles%20remontrances%20du%20parlement%20de%20Normandie%20%20%20au%20semestre%20de%20se%22%20and%20(century%20adj%20%2217%22)&ma

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22pitre%20h%C3%A9roque%20au%20roi%20sur%20sa%20premi%C3%A8re%20communion%20%20ou%20les%20Esp%C3%A9rances%20triomphantes%20du%20roi%20tr%C3%A8schr%C3%A9tien%20%20prises%20du%20premier%20et%20du%20plus%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22pouvantable%20l%20vision%20apparue%20sur%20labbaye%20de%20Marmoutiers%20lez%20Tours%20%20envoy%C3%A9e%20par%20une%20lettre%20dun%20bourgeois%20de%20la%20ville%20%C3%A0%20un%20sien%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Extraits%20des%20registres%20du%20Parlement%20%20contenant%20la%20harangue%20faite%20au%20roi%20et%20%C3%A0%20la%20reine%20r%C3%A9gente%20par%20M%20Talon%20%20avocat%20g%C3%A9n%C3%A9ral%20%20avec%20so%22%20and%20(century%20adj%20%221

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20circulaire%20envoy%C3%A9e%20par%20le%20roi%20%C3%A0%20tous%20les%20gouverneurs%20du%20royaume%20de%20France%20%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20circulaire%20et%20v%C3%A9ritable%20de%20l%20archiduc%20L%C3%A9opold%20envoy%C3%A9e%20%C3%A0%20tous%20les%20gouverneurs%20%20pr%C3%A9vts%20et%20%C3%A9chevins%20des%20villes%20et%20bourgs%20de%20Fr%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20circulaire%20pour%20la%20convocation%20des%20tats%20g%C3%A9n%C3%A9raux%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20contenant%20ce%20qui%20s%20est%20pass%C3%A9%20en%

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20d%20un%20bourgeois%20de%20Paris%20%20%C3%A9tant%20%C3%A0%20la%20cour%20%20envoy%C3%A9e%20%C3%A0%20Paris%20%C3%A0%20un%20sien%20ami%20%20le%2026%20janvier%201649%20%20sur%20le%20sujet%20des%20pr%C3%A9sents%20mouve%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20d%20un%20cardinal%20%C3%A0%20monsieur%20le%20cardinal%20Mazarin%20%20pour%20r%C3%A9ponse%20%C3%A0%20sa%20lettre%20du%2012%20d%C3%A9cembre%201654%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20d%20un%20cavalier%20%C3%A0%20sa%20matresse%20%20en%20vers%20burlesques%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20d%20un%20habitant%20de%20la%20ville%20de%20Blois%20%20%C3%A9crite%20%C3%A0%20un%20sien%20ami%20sur%20les%20d%C3%A9sordres%20%20pilleries%20%20sacril%C3%A9ges%20et%20violements%20que%20le%20cardi%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20d%20un%20inconnu%20envoy%C3%A9e%20%C3%A0%20un%20sien%20ami%20%C3%A0%20SaintGermainenLaye%20%20en%20vers%20burlesques%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20d%20un%20marchand%20de%20Li%C3%A8ge%20%C3%A0%20un%20sien%20correspondant%20de%20Paris%20%20avec%20l%20instruction%20secr%C3%A8te%20du%20cardinal%20Mazarin%20pour%20Zongo%20Ondedei%20%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operati

...

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20envoy%C3%A9e%20par%20l%20archiduc%20L%C3%A9opold%20%C3%A0%20monseigneur%20le%20duc%20d%20Orl%C3%A9ans%20%20avec%20la%20R%C3%A9ponse%20de%20Son%20Altesse%20Royale%20sur%20le%20sujet%20de%20la%20paix%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20envoy%C3%A9e%20sur%20le%20sujet%20de%20l%20assembl%C3%A9e%20de%20la%20noblesse%20%20et%20des%20procurations%20%C3%A9crites%20dans%20les%20provinces%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20et%20d%C3%A9claration%20du%20roi%20%20avec%20les%20articles%20en%20cons%C3%A9quence%20accord%C3%A9s%20par%20Sa%20Majest%C3%A9%20pour%20le%20repos%20et%20pour%20la%20tranquillit%C3%A9%20publiq%22%20and%20(century%20adj%20%2217%22)&maximumR

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20d%20Ariste%20%C3%A0%20Nicandre%20sur%20la%20bataille%20de%20Rethel%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20de%20d%C3%A9claration%20du%20roi%20%20du%20mois%20de%20juin%201649%20%20par%20lesquelles%20Sa%20Majest%C3%A9%20%20pour%20reconnoissance%20des%20services%20et%20fid%C3%A9lit%C3%A9%20%C3%A0%20elle%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20de%20deux%20amis%20sur%20la%20prise%20de%20la%20Bastille%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20de%20la%20cour%20de%20Parlement%20de%20Bordeaux%20%C3%A9crites%20au%20Parlement%20de%20Paris%20%20sur%20

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20et%20arr%C3%AAts%20de%20la%20cour%20de%20Parlement%20de%20Normandie%20envoy%C3%A9s%20%C3%A0%20la%20cour%20de%20Parlement%20de%20Paris%20pour%20l%20adjonction%20%20desdites%20cou%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20et%20arr%C3%AAts%20pour%20la%20jonction%20des%20parlements%20du%20royaume%20et%20affaires%20pr%C3%A9sentes%20%20et%20la%20tr%C3%A8s%20humble%20remontrance%20du%20Parlement%20au%20r%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettres%20et%20d%C3%A9claration%20du%20roi%20sur%20le%20sujet%20de%20sa%20sortie%20de%20Paris%20%20avec%20l%20arr%C3%AAt%20de%20son%20conseil%20d%20tat%20%20portant%20cassation%20de%20celui%20du%22%20and%20(century%20adj%20%2217%22)

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Seconde%20lettre%20du%20sieur%20Du%20Pelletier%20%C3%A0%20monseigneur%20le%20duc%20de%20Beaufort%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Seconde%20lettre%20%C3%A9crite%20%C3%A0%20messieurs%20du%20Parlement%20par%20M%20le%20prince%20de%20Cond%C3%A9%20%20%20servant%20de%20r%C3%A9ponse%20%C3%A0%20l%20%C3%A9crit%20envoy%C3%A9%20par%20la%20reine%20r%C3%A9gente%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Sentence%20de%20M%20le%20pr%C3%A9vt%20de%20Paris%20portant%20condamnation%20d%20un%20libelle%20intitul%C3%A9%20Lettre%20de%20monseigneur%20l%20minentissime%20cardinal%20de%20Ret%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=

https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20du%20mar%C3%A9chal%20de%20Turenne%20%C3%A0%20la%20reyne%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20du%20roy%20%C3%A9crite%20%C3%A0%20son%20parlement%20de%20Paris%20sur%20les%20affaires%20pr%C3%A9sentes%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20du%20roy%20envoy%C3%A9e%20%C3%A0%20messieurs%20de%20la%20Chambre%20des%20comptes%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20dc.title%20any%20%22Lettre%20du%20roy%20envoy%C3%A9e%20%C3%A0%20monsieur%20le%20mar%C3%A9chal%20de%20Lhospital%20%20gouverneur%20de%20la%20ville%20de%20Paris%22%20and%20(century%20adj%20%2217%22)&maximumRecords=5
htt

**Enregistrement automatique des PDFs à partir des résultats de l'API si, et seulement si les titres des résultats de l'API enregistrés respectent la regex**

In [8]:
p = re.compile("1648|1649|1650|1651|1652|1653",re.IGNORECASE)

In [16]:
import glob

def enregistrement_lien():
    for filepath in glob.iglob('*.xml'):
        try:
            if filepath.endswith('.xml'):
                print(filepath)
                tree = etree.parse("%s" %filepath)
                #Récupération du pdf de la 1ère sortie de l'api si le titre commence par "Lett"
                for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[1]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                    m = p.search(date.text)
                    if m:
                        print('Match found: ', m.group())
                        for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[1]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                            print ("TITRE: "+title.text)
                        for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[1]/srw:extraRecordData[1]/link[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/'}):
                            url_pdf=link.text+".pdf"
                            print("Lien vers le pdf disponible [1]:")
                            print(url_pdf)
                    else:
                        print('No match')
                    for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[2]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                        m = p.search(date.text)
                        if m:
                            print('Match found: ', m.group())
                            for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[2]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                                print ("TITRE: "+title.text)
                                for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[2]/srw:extraRecordData[1]/link[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/'}):
                                    url_pdf=link.text+".pdf"
                                    print("Lien vers le pdf disponible [2]:")
                                    print(url_pdf)
                        else:
                            print('No match')
                    for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[3]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}): 
                        m=p.search(date.text)
                        if m:
                            print('Match found: ', m.group())
                            for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[3]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                                print ("TITRE: "+title.text)
                                for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[3]/srw:extraRecordData[1]/link[1]",
                                                                           namespaces={'srw': 'http://www.loc.gov/zing/srw/'}):
                                    url_pdf=link.text+".pdf"
                                    print("Lien vers le pdf disponible [3]:")
                                    print(url_pdf)
                    else:
                        print('No match')
                    for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[4]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                        m = p.search(date.text)
                        if m:
                            print('Match found: ', m.group())
                            for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[4]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                                print ("TITRE: "+title.text)
                                for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[4]/srw:extraRecordData[1]/link[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/'}): 
                                    url_pdf=link.text+".pdf"
                                    print("Lien vers le pdf disponible [4]:")
                                    print(url_pdf)
                    else:
                        print('No match')
                    for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[5]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                        m = p.search(date.text)
                        if m:
                            print('Match found: ', m.group())
                            for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[5]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                                print ("TITRE: "+title.text)
                                for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[5]/srw:extraRecordData[1]/link[1]",
                                namespaces={'srw': 'http://www.loc.gov/zing/srw/'}):
                                    url_pdf=link.text+".pdf"
                                    print("Lien vers le pdf disponible [5]:")
                                    print(url_pdf)
                    else:
                        print('No match')
        except:
            pass

In [17]:
enregistrement_lien()

1018.xml
No match
No match
No match
No match
No match
1021.xml
No match
No match
Match found:  1651
TITRE: Lettre d'un particulier sur la sortie de MM. les Princes (par François Davenne)
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/btv1b8606947x.pdf
No match
No match
No match
1067.xml
No match
No match
No match
No match
No match
109s1.xml
No match
No match
No match
No match
No match
110s1.xml
Match found:  1649
TITRE: Lettre a monsieur le cardinal, burlesque.
Lien vers le pdf disponible [1]:
https://gallica.bnf.fr/ark:/12148/bpt6k5542391s.pdf
Match found:  1649
TITRE: L'adieu de Mazarin . Burlesque
Lien vers le pdf disponible [2]:
https://gallica.bnf.fr/ark:/12148/bpt6k55764575.pdf
Match found:  1649
TITRE: Remontrance à Monsieur le cardinal. Burlesque
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/bpt6k5425931q.pdf
No match
Match found:  1649
TITRE: Lettre du vray soldat françois au cavalier Georges : en suitte de la lettre à Mr le Cardinal, b

No match
No match
Match found:  1651
TITRE: [monnaie : Avignon, 1651]
Lien vers le pdf disponible [5]:
https://gallica.bnf.fr/ark:/12148/btv1b11332963k.pdf
No match
1875.xml
No match
Match found:  1652
TITRE: Lettre prophetique, sur les affaires du temps. Presentée à Messieurs les Princes.
Lien vers le pdf disponible [2]:
https://gallica.bnf.fr/ark:/12148/bpt6k857020f.pdf
Match found:  1649
TITRE: Dialogue de deux guepeins, sur les affaires du temps.
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/bpt6k130783z.pdf
No match
No match
No match
1876.xml
No match
No match
No match
No match
No match
1877.xml
Match found:  1649
TITRE: La résipiscence du cardinal Mazarin
Lien vers le pdf disponible [1]:
https://gallica.bnf.fr/ark:/12148/bpt6k5425500s.pdf
Match found:  1649
TITRE: Recueil de quelques pièces contre le Cardinal Mazarin Au Cardinal Mazarin
Lien vers le pdf disponible [2]:
https://gallica.bnf.fr/ark:/12148/bpt6k5426104c.pdf
Match found:  1649
TITRE: A Mazarin , s

Lien vers le pdf disponible [4]:
https://gallica.bnf.fr/ark:/12148/bpt6k5695178s.pdf
No match
No match
1964.xml
No match
No match
No match
No match
No match
1965.xml
No match
Match found:  1650
TITRE: Lettre de Messieurs du Parlement de Bourdeaux, presentée le samedy 6. aoust 1650. A Messieurs du Parlement de Paris ; Par messieurs les Deputez dudit Parlement de Bourdeaux : sur le sujet de la continuation des violences du sieur duc d'Espernon, protégé par Monsieur le cardinal Mazarin.
Lien vers le pdf disponible [2]:
https://gallica.bnf.fr/ark:/12148/bpt6k375125j.pdf
Match found:  1650
TITRE: Plan de La Bassée comme elle est à présent fortiffié // 1650 : [dessin]
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/btv1b69021907.pdf
No match
Match found:  1650
TITRE: Plan de la ville de Péronne en Picardie comme elle est [?] // 1650 : [dessin]
Lien vers le pdf disponible [4]:
https://gallica.bnf.fr/ark:/12148/btv1b6902473r.pdf
No match
Match found:  1649
TITRE: Stances au 

No match
No match
No match
No match
No match
2047.xml
No match
No match
No match
No match
No match
2048.xml
Match found:  1650
TITRE: Journal des délibérations tenues en Parlement, toutes les Chambres assemblées, & à l'Hostel d'Orleans depuis le 5. jour d'Aoust 1650. jusques à présent, ou ont assisté Monseigneur le Duc d'Orleans, Messieurs de Beaufort, de l'hospital, de Brissac, & le Coadjuteur : Touchant l'éloignement du Cardinal Mazarin, la Guerre de Bourdeaux, & l'affaire de Messieurs les Princes : Avec les Harangues faites sur ce sujet par Messieurs les Presidents & Conseillers, & les Arrests donnez en consequence
Lien vers le pdf disponible [1]:
https://gallica.bnf.fr/ark:/12148/btv1b8606944p.pdf
Match found:  1652
TITRE: Relation véritable de ce qui s'est fait et passé en Parlement, toutes les chambres assemblées, en présence de Son Altesse royale et plusieurs ducs et pairs de France, avec la nouvelle déclaration de Son Altesse royale pour la paix générale
Lien vers le pdf dispon

No match
Match found:  1650
TITRE: Lettre en vers à Son Altesse Mademoiselle de Longueville.
Lien vers le pdf disponible [5]:
https://gallica.bnf.fr/ark:/12148/cb32806070n/date.pdf
No match
2109.xml
No match
No match
No match
No match
No match
2110.xml
No match
No match
No match
No match
No match
2111.xml
No match
No match
Match found:  1651
TITRE: A Son Altesse royale, Monseigneur le prince de Galles . Ode
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/bpt6k5568231k.pdf
No match
No match
No match
2112.xml
No match
No match
Match found:  1649
TITRE: Ode au Roy sur les mouvemens arrivez à Paris, au commencement de l'année 1649, ensuite de l'enlèvement fait de Sa Majesté, par le Cardinal Mazarin
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/bpt6k5711697n.pdf
No match
No match
No match
2113.xml
Match found:  1649
TITRE: A Mazarin , sonnet
Lien vers le pdf disponible [1]:
https://gallica.bnf.fr/ark:/12148/bpt6k5657474p.pdf
No match
No match
Match fo

No match
No match
No match
No match
No match
2227.xml
No match
No match
No match
No match
Match found:  1648
TITRE: Le vray théâtre d'honneur et de chevalerie, ou le Miroir héroique de la noblesse , contenant les combats ou jeux sacrez des Grecs et des Romains, les triomphes, les tournois, les joustes, les pas, les emprises ou entreprises, les armes, les combats à la barrière, les carrosels, les courses de bague et de la quintaine, les machines, les chariots de triomphe, les cartels, les devises, les prix, les voeux, les sermens, les cérémonies, les statuts, les ordres et autres magnificences et exercices des anciens nobles durant la paix. Avec le formulaire d'un tournoy tel qu'on le pourroit faire à présent avec les armes dont les gentils-hommes se servent à la guerre... Par Marc de Wlson, sieur de La Colombière,...
Lien vers le pdf disponible [5]:
https://gallica.bnf.fr/ark:/12148/bpt6k480023j.pdf
No match
2228.xml
Match found:  1649
TITRE: Eloges du coeur royal de Mgr. le duc de Bea

No match
No match
No match
No match
No match
415.xml
No match
No match
No match
No match
Match found:  1650
TITRE: [monnaie : Orange, 1650]
Lien vers le pdf disponible [5]:
https://gallica.bnf.fr/ark:/12148/btv1b113329730.pdf
No match
49s2.xml
Match found:  1649
TITRE: Plan de la ville de Lion // 1649 : [dessin] / Beaulieu
Lien vers le pdf disponible [1]:
https://gallica.bnf.fr/ark:/12148/btv1b6902325b.pdf
No match
Match found:  1649
TITRE: Plan de la ville et port de Marseille, en Provence, 1649 : [dessin] / Beaulieu
Lien vers le pdf disponible [3]:
https://gallica.bnf.fr/ark:/12148/btv1b69017151.pdf
No match
Match found:  1649
TITRE: Plan d'Albe, en Milanais en Milanois avec ses fortifications. 1649 : [dessin] / Beaulieu
Lien vers le pdf disponible [4]:
https://gallica.bnf.fr/ark:/12148/btv1b6903580j.pdf
No match
No match
50s2.xml
No match
No match
No match
No match
No match
51s2.xml
Match found:  1649
TITRE: Lettre a monsieur le cardinal, burlesque.
Lien vers le pdf disponible [1]:


**Statistiques sur les résultats obtenus**

In [19]:
%%capture cap --no-stderr
enregistrement_lien()

with open('sortie\output_epistolaire.txt', 'w',encoding='utf-8') as f:
    f.write(cap.stdout)
    f.close()

#Lancer deux fois la case pour qu'elle fonctionne

In [20]:
texte = ""
with open("sortie\output_epistolaire.txt", "r", encoding='utf-8') as f:
    texte = texte.join(line.rstrip("\n") + " " for line in f.readlines())

In [21]:
p = re.compile('.xml',re.IGNORECASE)
res=p.finditer(texte)
start_pattern=[m.start() for m in res]
print("Nombre de fichiers xml exportés (=nombre de titres interrogés sur l'API):")
print(len(start_pattern))

#Cherche le nombre de fichiers vides
p = re.compile('.xml\s(1|2)',re.IGNORECASE)
res=p.finditer(texte)
start_pattern=[m.start() for m in res]
print("Nombre de fichiers xml vides (requête API sans résultat ou API dysfonctionnelle):")
print(len(start_pattern))

#Cherche le nombre de requêtes retournées comprenant "Lett"(avec ou sans lien)
p = re.compile('Match found:',re.IGNORECASE)
res=p.finditer(texte)
start_pattern=[m.start() for m in res]
print("Nombre de requêtes retournées comprenant la regex:")
print(len(start_pattern))

#Cherche le nombre de liens trouvés
p = re.compile('Lien vers le pdf disponible',re.IGNORECASE)
res=p.finditer(texte)
start_pattern=[m.start() for m in res]
print("Nombre de liens URL trouvés:")
print(len(start_pattern))

Nombre de fichiers xml exportés (=nombre de titres interrogés sur l'API):
584
Nombre de fichiers xml vides (requête API sans résultat ou API dysfonctionnelle):
0
Nombre de requêtes retournées comprenant la regex:
436
Nombre de liens URL trouvés:
436
